In [ ]:
!pip install langchain
!pip install pytube
!pip install sentence_transformers
!pip install xformers
!pip install bitsandbytes accelerate transformers
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-mei4s81o
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-mei4s81o
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from transformers import AutoTokenizer, TextStreamer,AutoModelForCausalLM, pipeline
import os
import sys
import torch
import whisper
from pytube import YouTube

In [ ]:
video_link = "https://www.youtube.com/watch?v=mKDeMvFjiQ4"

try:
    video = YouTube(video_link)
    audio = video.streams.filter(only_audio=True, file_extension='mp4').first()
    audio.download(filename="audio-file.mp4")
    print('Download Completed!')

except:
    print("Connection Error")

Download Completed!


In [ ]:
model = whisper.load_model("base")
result = model.transcribe("audio-file.mp4")

In [ ]:
print(result["text"])

 This is the story of Bill and Katie, who are the best friends. Last year Bill was working in a big company, but he wasn't very happy with his boss. And Katie, she was really bored with working as a waitress. She had to take a long way to work. Best friend Julie and they decided to make a killer explainer video together. They started looking for actors to pick the right one. Now that they have the character, they need the right setting. The website is for their coffee shop, so they go into the city. As they walk through the streets, they see the perfect little coffee shop to film. Oh no, there's a problem. Zombies invaded the city and the police can't stop them. Mike and Julia doomed. They run away and forget about their video. So this is Bill. He belongs to the Stone Cutters Club, a group of business people who do good for the community. Now every year, the stone cutters put on a dinner and charity dance for people in need. Bill signed up with the Cause Guru website, so the stone cutt

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=32)
text_chunks = text_splitter.split_text(result["text"])
len(text_chunks)

6

In [ ]:
print(text_chunks[0])

This is the story of Bill and Katie, who are the best friends. Last year Bill was working in a big company, but he wasn't very happy with his boss. And Katie, she was really bored with working as a waitress. She had to take a long way to work. Best friend


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
query_result = embeddings.embed_query("Hello World")

In [ ]:
print("Length", len(query_result))

Length 384


In [ ]:
print(query_result)

[-0.03447728976607323, 0.031023146584630013, 0.006734955124557018, 0.02610897459089756, -0.03936200961470604, -0.16030250489711761, 0.06692398339509964, -0.006441479083150625, -0.04745052754878998, 0.014758859761059284, 0.07087533921003342, 0.05552750825881958, 0.01919332891702652, -0.02625131607055664, -0.010109510272741318, -0.026940539479255676, 0.02230740711092949, -0.022226667031645775, -0.1496926248073578, -0.017493100836873055, 0.007676260080188513, 0.05435231700539589, 0.003254511160776019, 0.031725987792015076, -0.08462139219045639, -0.029406022280454636, 0.05159567669034004, 0.048124074935913086, -0.003314799861982465, -0.05827919393777847, 0.04196930676698685, 0.022210730239748955, 0.1281888335943222, -0.02233896404504776, -0.011656232178211212, 0.06292839348316193, -0.03287629038095474, -0.09122603386640549, -0.03117532655596733, 0.05269954726099968, 0.0470348559319973, -0.08420305699110031, -0.030056172981858253, -0.020744824782013893, 0.009517737664282322, -0.003721745917

In [ ]:
print(type(query_result))

<class 'list'>


In [ ]:
!pip install pinecone-client

In [ ]:
# Import the Pinecone library
from pinecone import Pinecone

# Set your environment variables
os.environ['PINECONE_API_KEY'] = '4578e5a0-4444-480a-8040-3b114c0bb9a8'
os.environ['PINECONE_ENVIRONMENT'] = 'gcp-starter'
api_key = "4578e5a0-4444-480a-8040-3b114c0bb9a8"
environment = "gcp-starter"
# Initialize the Pinecone client
pc = Pinecone(api_key=api_key,environment=environment)

In [ ]:
indexes = pc.list_indexes()
print(indexes)

{'indexes': [{'dimension': 384,
              'host': 'doc-chat-ef51ae3.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'doc-chat',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [ ]:
index = pc.Index('doc-chat')
index_name='doc-chat'

In [ ]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 6e-05,
 'namespaces': {'': {'vector_count': 6}},
 'total_vector_count': 6}

In [ ]:
embedding_list=[]
for text in text_chunks:
  embedding_list.append(embeddings.embed_query(text))

In [ ]:
print(embedding_list)
print(len(embedding_list))

[[-0.05193227902054787, 0.014974727295339108, 0.06725891679525375, -0.013096134178340435, -0.011082282289862633, 0.02363680861890316, -0.03374293074011803, -0.01082273107022047, -0.07074625045061111, -0.10477222502231598, 0.046646688133478165, 0.10294545441865921, 0.04354002699255943, 0.05900841951370239, -0.0692569762468338, -0.024105796590447426, -0.0005542692961171269, -0.05520312488079071, -0.04703099653124809, -0.014311602339148521, -0.007753225043416023, -0.043173160403966904, -0.03892286866903305, 0.03640507161617279, 0.023123161867260933, 0.05235898494720459, 0.05489184707403183, -0.06009134277701378, -0.07424572110176086, 0.0206332765519619, -0.0745956152677536, 0.01524308416992426, -0.024404127150774002, 0.004960023798048496, 0.02376175858080387, 0.008372113108634949, 0.05536484718322754, 0.0473916232585907, 0.0551232174038887, -0.0064967041835188866, 0.002721871482208371, -0.11362801492214203, 0.03113035298883915, -0.0457809679210186, -0.04857645183801651, 0.0184462461620569

In [ ]:
data = []
for i, text in enumerate(text_chunks):
  data.append({
      "id":str(i),
      "values": embedding_list[i],
      "metadata" : {"text" : text}
  })

print(data[0])
# Upload data to Pinecone
# index.upsert(data)

{'id': '0', 'values': [-0.05193227902054787, 0.014974727295339108, 0.06725891679525375, -0.013096134178340435, -0.011082282289862633, 0.02363680861890316, -0.03374293074011803, -0.01082273107022047, -0.07074625045061111, -0.10477222502231598, 0.046646688133478165, 0.10294545441865921, 0.04354002699255943, 0.05900841951370239, -0.0692569762468338, -0.024105796590447426, -0.0005542692961171269, -0.05520312488079071, -0.04703099653124809, -0.014311602339148521, -0.007753225043416023, -0.043173160403966904, -0.03892286866903305, 0.03640507161617279, 0.023123161867260933, 0.05235898494720459, 0.05489184707403183, -0.06009134277701378, -0.07424572110176086, 0.0206332765519619, -0.0745956152677536, 0.01524308416992426, -0.024404127150774002, 0.004960023798048496, 0.02376175858080387, 0.008372113108634949, 0.05536484718322754, 0.0473916232585907, 0.0551232174038887, -0.0064967041835188866, 0.002721871482208371, -0.11362801492214203, 0.03113035298883915, -0.0457809679210186, -0.0485764518380165

In [ ]:
!pip install --upgrade --quiet  langchain-pinecone
from langchain_pinecone import PineconeVectorStore



In [ ]:
index.query(
    vector=embeddings.embed_query("Bill and Katie"),
    top_k=3
)

{'matches': [{'id': '0', 'score': 0.581930816, 'values': []},
             {'id': '3', 'score': 0.396250963, 'values': []},
             {'id': '1', 'score': 0.270754218, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone.from_existing_index(
    index_name, embeddings, text_field
)


In [ ]:
docsearch = PineconeVectorStore.from_existing_index(embedding=embeddings, index_name=index_name)

In [ ]:
print(type(vectorstore))
print(vectorstore)
print(type(docsearch))
print(docsearch)


<class 'langchain_community.vectorstores.pinecone.Pinecone'>
<class 'langchain_pinecone.vectorstores.PineconeVectorStore'>


In [ ]:
query = "What is the story about ?"
res = docsearch.similarity_search(query,k=2)
for r in res:
  print(r.page_content)


This is the story of Bill and Katie, who are the best friends. Last year Bill was working in a big company, but he wasn't very happy with his boss. And Katie, she was really bored with working as a waitress. She had to take a long way to work. Best friend
a long way to work. Best friend Julie and they decided to make a killer explainer video together. They started looking for actors to pick the right one. Now that they have the character, they need the right setting. The website is for their coffee shop,


In [ ]:
from huggingface_hub import notebook_login
import torch

In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

In [ ]:
SYSTEM_PROMPT ="""Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer about the context. If the question is in general go ahead and answer it."""


In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [ ]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [ ]:
instruction = """
{context}

Question: {question}
"""

In [ ]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [ ]:
template

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer about the context. If the question is in general go ahead and answer it.\n<</SYS>>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
result = qa_chain("What is the story about?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
result

{'query': 'What is the story about?',
 'result': "[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer about the context. If the question is in general go ahead and answer it.\n<</SYS>>\n\n\nThis is the story of Bill and Katie, who are the best friends. Last year Bill was working in a big company, but he wasn't very happy with his boss. And Katie, she was really bored with working as a waitress. She had to take a long way to work. Best friend\n\na long way to work. Best friend Julie and they decided to make a killer explainer video together. They started looking for actors to pick the right one. Now that they have the character, they need the right setting. The website is for their coffee shop,\n\nand Julia doomed. They run away and forget about their video. So this is Bill. He belongs to the Stone Cutters Club, a group of business people who do good for the comm

In [ ]:
while True:
    user_input=input(f"prompt:")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa_chain({'query':user_input})
    print(f"Answer:{result['result'].split('[/INST]')[1].strip()}")

prompt:Hello
Answer:Hello! I'm here to help you with any questions you may have. Based on the context you provided, it seems like you're working on a story or script of some kind. Is there a specific question or topic you'd like me to help with?
prompt:Can you list down the characters ?
Answer:Sure, here are the characters mentioned in the context:

1. Bill
2. Katie
3. Julie (Bill and Katie's best friend)
4. The boss (Bill's boss at the big company)
5. The waitress (Katie's job)
6. The Stone Cutters Club (a group of business people who do good for the community)
7. The volunteers (managed by the Stone Cutters Club through the Cause Guru website)
prompt:What work does Bill do ?
Answer:Based on the context provided, Bill works for the Stone Cutters Club, a group of business people who do good for the community.
prompt:What is it they are trying to do ?
Answer:Based on the context provided, it seems that Bill and the Stone Cutters Club are organizing a charity dinner and dance event to ra

KeyboardInterrupt: Interrupted by user